In [1]:
import pandas as pd
import numpy as np
import math

In [30]:
targetAttribute = 'Poisonous/Edible'
minimalGainThreshold = 1e-6
#targetAttribute = ''
targetValue = []

In [3]:
# data class to hold csv data
def entropy(df):
    #target unique value calculation in form of dictionary 
    targetDict = pd.value_counts(df[targetAttribute]).to_dict()
    dfLength =len(df)
    
    totalEntropy = 0
    #print(targetDict)
    for key, val in targetDict.items():
        multipleVal = val/dfLength
        totalEntropy += multipleVal * math.log2(multipleVal)
    
    return 0-totalEntropy

In [4]:
#add values to targetValue
def initializeList(data):
    groupedData = data.groupby([targetAttribute]).groups.keys()
    return groupedData
    

In [5]:
def calcInforGainForEachAtrribute(df):
    # return maximum info gain
    maxInfoGain =0;
    infoGainDict = {}
    attributes = list(df)
    totalEntropy = entropy(df)

    for att in attributes:
        #don't calculate gain for target attribute
        if (att ==targetAttribute ):
            continue
        grouped_data = df.groupby([att], as_index=False)
        totalRows = len(df)
        subEntropy = 0
        #print('keys for ', att, ' total- ', totalRows)
        
        for key, value in grouped_data:
            eachGroupRows = len(value)
            #print('key-', key, ' eachGroupRows- ', eachGroupRows)
            
            S = entropy(value)
            
            #calculate |Sv/S|
            valEntrpy = eachGroupRows / totalRows
            
            #calculate |Sv/S|*Entropy(Sv)
            subEntropy = subEntropy + valEntrpy * S

        individualEntrpyGain = totalEntropy - subEntropy
        infoGainDict[att] = individualEntrpyGain
        #print('gain-', att, ' - ', individualEntrpyGain)
        if (individualEntrpyGain > maxInfoGain):
            maxInfoGain = individualEntrpyGain
            maxGainAttr = att;
    
    # If there's no gain at all, nothing has to be done, just return the original set
    if (maxInfoGain < 1e-6):
        #print("in maxInfoGain")
        #print(maxInfoGain,maxGainAttr)
        return None, None
    else:
        return maxGainAttr,maxInfoGain



In [6]:
def trueSet(s):
    return len(set(s)) == 1

def createTree_proto(df):
    
    x = df.drop(labels=targetAttribute, axis=1)
    y = df[targetAttribute]
    
    # If there could be no split, just return the original set
    if trueSet(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest mutual information
    
    maxGainAttr,maxInfoGain = calcInforGainForEachAtrribute(df)
    
    if((maxGainAttr== None) and (maxInfoGain == None)):
        return y

    # We split using the selected attribute
    sets = df.groupby(maxGainAttr)
    print(sets)
    for key,value in sets:
        print(key)
        
    print(type(sets))
    res = {}
    for k, v in sets:
        res["x_%s = %s" % (maxGainAttr, k)] = createTree(v)

    return res



In [21]:
def trueSet(s):
    return len(set(s)) == 1

def createTree(df):
    
    x = df.drop(labels=targetAttribute, axis=1)
    y = df[targetAttribute]
    
    # If there could be no split, just return the original set
    if trueSet(y) or len(y) == 0:
        return y

    # We get attribute that gives the highest mutual information
    
    maxGainAttr,maxInfoGain = calcInforGainForEachAtrribute(df)
    
    if((maxGainAttr== None) and (maxInfoGain == None)):
        return y

    # We split using the selected attribute
    sets = df.groupby(maxGainAttr)
    #print(sets)
    
    #print(type(sets))
    res = {}
    for k, v in sets:
        res["x_%s = %s" % (maxGainAttr, k)] = createTree(v)

    return res


In [26]:
from sklearn.model_selection import train_test_split

In [38]:
datasetAfterProcessing = pd.read_excel('MushroomDataSet_After_PreProcessing.xlsx')
X = datasetAfterProcessing.drop(labels=targetAttribute, axis=1)
y = datasetAfterProcessing[targetAttribute]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

print(type(X_train))

(6499, 22) (6499,)
(1625, 22) (1625,)
<class 'pandas.core.frame.DataFrame'>


In [40]:
def main():
    #df = pd.read_excel("MushroomDataSet_After_PreProcessing.xlsx")
    
    #datasetAfterProcessing = pd.read_excel('MushroomDataSet_After_PreProcessing.xlsx')
    #split_value = (int) (len(datasetAfterProcessing.index)*0.8)
    #training_data = pd.DataFrame(datasetAfterProcessing.iloc[:split_value,:].values)
    #training_data.columns= ['Poisonous/Edible','cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring	','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color',	'population','habitat']
    
    datasetAfterProcessing = pd.read_excel('MushroomDataSet_After_PreProcessing.xlsx')
    X = datasetAfterProcessing.drop(labels=targetAttribute, axis=1)
    y = datasetAfterProcessing[targetAttribute]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)
    
    X_train[targetAttribute] = y_train
    targetValue = initializeList(X_train)
    
    
    print(createTree(X_train))
    
    
if __name__ == "__main__": main()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'x_odor = a': 1662    e
322     e
1467    e
1514    e
838     e
121     e
196     e
415     e
880     e
1784    e
267     e
303     e
527     e
755     e
1201    e
501     e
1513    e
216     e
421     e
347     e
333     e
259     e
45      e
1298    e
426     e
410     e
210     e
652     e
864     e
439     e
       ..
1931    e
635     e
876     e
574     e
784     e
154     e
569     e
301     e
815     e
562     e
1045    e
98      e
623     e
262     e
202     e
709     e
863     e
206     e
870     e
804     e
379     e
161     e
1021    e
775     e
646     e
1528    e
189     e
1184    e
769     e
466     e
Name: Poisonous/Edible, Length: 320, dtype: object, 'x_odor = c': 2860    p
3656    p
3047    p
3400    p
3393    p
3616    p
3887    p
3833    p
3994    p
4098    p
2540    p
3742    p
3882    p
3181    p
3869    p
3169    p
3881    p
3792    p
3692    p
3872    p
3454    p
2627    p
3365    p
3852    p
3443    p
3182    p
4198    p
3739    p
3209    p
3885    p
       ..